In [8]:
import numpy as np
from collections import defaultdict
import random
class Agent:

    def __init__(self, epsilon , alpha , gamma, nA = 6):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        #self.env = env
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma

    def select_action(self, state, i_episode, eps_decay = 0.999, eps_min = 0.0005):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        #self.epsilon -= 0.0005 
        #self.alpha -= 0.0002
        #self.gamma -= -.0002
        self.epsilon = max( self.epsilon*eps_decay , eps_min)
        
        
        if random.random() > self.epsilon:
            return (np.argmax(self.Q[state]))
        else:
            return np.random.choice(np.array(self.nA))
            #return self.env.action_space.sample()
       # return np.random.choice(self.nA)

    def step(self, state, action, reward, done , next_state = None, next_action = None ):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        current = self.Q[state][action]
        Q_s = self.Q[next_state][next_action] if next_state is not None else 0
        target = reward + self.gamma * Q_s
        new_value = current + self.alpha * (target - current)
        self.Q[state][action] = new_value
        
        
        
        #self.Q[state][action] += 1

In [9]:
from collections import deque
import sys
import math
import numpy as np

def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.
    
    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v1 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        action = agent.select_action(state, i_episode)
        
        while True:
              
            # agent performs the selected action
            next_state, reward, done, _ = env.step(action)
            samp_reward += reward
            if not done:
                
                next_action = agent.select_action(next_state, i_episode)
            
            # agent performs internal updates based on sampled experience
                agent.step(state, action, reward, done , next_state, next_action )
            
                state = next_state
                action = next_action
            # update the sampled reward
            #samp_reward += reward
            # update the state (s <- s') to next time step
            #state = next_state
            if done:
                agent.step(state, action, reward, done, next_state = None, next_action = None )
                
                # save final sampled reward
                samp_rewards.append(samp_reward)
                break
        if (i_episode >= 100):
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')
    return avg_rewards, best_avg_reward

In [10]:
#from agent import Agent
#from monitor import interact
import gym
import numpy as np

env = gym.make('Taxi-v2')
agent = Agent(epsilon = 0.9, alpha = 0.02, gamma = 0.9)
avg_rewards, best_avg_reward= interact(env, agent, num_episodes=100000)

Episode 97249/100000 || Best average reward 9.4718